# Ajuste de Hiperparámetros

*La terminología específica de ML puede diferir de otros campos.*

- **Hiperparámetros**: Valores que configuran el entrenamiento del modelo, distintos de los parámetros aprendidos de los datos (ej. tasa de regularización en regresión logística, tasa de aprendizaje en redes neuronales convolucionales).

- **Ajuste de Hiperparámetros**: Proceso de encontrar los mejores valores de hiperparámetros para un modelo dado, buscando optimizar una métrica de rendimiento (ej. precisión).

Pasos en Azure ML:

1. **Definir espacio de búsqueda**: Rango de valores posibles para cada hiperparámetro.
2. **Configurar muestreo**: Método de selección de combinaciones de hiperparámetros para probar.
3. **Seleccionar política de terminación temprana**: Detener experimentos con bajo rendimiento.
4. **Ejecutar experimento**: Entrenar modelos con diferentes combinaciones de hiperparámetros, evaluar su rendimiento y seleccionar el mejor.



Los modelos de ML aprenden a predecir valores desconocidos (etiquetas) para datos nuevos basados en relaciones entre valores conocidos (etiquetas) y características presentes en los datos de entrenamiento. Dependiendo del algoritmo utilizado, puede ser necesario especificar hiperparámetros para configurar el entrenamiento del modelo. Estos son distintos de los parámetros aprendidos de los datos.

El ajuste de hiperparámetros busca encontrar los mejores valores de estos parámetros para un modelo específico, optimizando una métrica de rendimiento.

#### 1. Definiendo el espacio de búsqueda para ajuste de hiperparámetros

El espacio de búsqueda es el conjunto de valores de hiperparámetros que se prueban durante el ajuste. Su definición depende del tipo de hiperparámetro:

- **Discretos**: Requieren valores específicos de un conjunto limitado. Se definen mediante una lista de Python, un rango o valores separados por comas. También se pueden elegir de distribuciones discretas predefinidas.
  - qnormal
  - quniform
  - qlognormal
  - qloguniforme
- **Continuos**: Pueden tomar cualquier valor dentro de un rango. Se definen mediante distribuciones continuas predefinidas.
  - normal
  - uniform
  - lognormal
  - loguniforme

Para definir el espacio de búsqueda, se crea un diccionario con expresiones para cada hiperparámetro. 
  Por ejemplo, podemos especificar que el tamaño de lote sea 16, 32 o 64, y la tasa de aprendizaje pueda tomar cualquier valor de una distribución normal con media 10 y desviación estándar 3.

#### 2. Configuración del muestreo

Los valores específicos utilizados en una ejecución de ajuste de hiperparámetros dependen del tipo de muestreo utilizado.

##### Muestreo de cuadrícula
El muestreo de cuadrícula sólo puede emplearse cuando todos los hiperparámetros son discretos, y se utiliza para probar todas las combinaciones posibles de parámetros en el espacio de búsqueda.

Por ejemplo, en el siguiente ejemplo de código, el muestreo de cuadrícula se utiliza para probar todas las combinaciones posibles del valor discreto de `batch_size` y `learning_rate`:

In [ ]:
from azureml.train.hyperdrive import GridParameterSampling, choice

param_space = {
                 '--batch_size': choice(16, 32, 64),
                 '--learning_rate': choice(0.01, 0.1, 1.0)
              }

param_sampling = GridParameterSampling(param_space)

##### Muestreo aleatorio

El muestreo aleatorio se usa para seleccionar aleatoriamente un valor para cada hiperparámetro, que puede ser una combinación de valores discretos y continuos

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, choice, normal

param_space = {
                 '--batch_size': choice(16, 32, 64),
                 '--learning_rate': normal(10, 3)
              }

param_sampling = RandomParameterSampling(param_space)

##### Muestreo bayesiano

El muestreo bayesiano elige valores de hiperparámetros basados en el algoritmo de optimización bayesiana, que intenta seleccionar combinaciones de parámetros que darán como resultado un mejor rendimiento de la selección anterior.

Solo puede usar el muestreo bayesiano con expresiones de parámetros choice, uniform y quniform, y no puede combinarlo con una directiva de terminación anticipada.

In [ ]:
from azureml.train.hyperdrive import BayesianParameterSampling, choice, uniform

param_space = {
                 '--batch_size': choice(16, 32, 64),
                 '--learning_rate': uniform(0.05, 0.1)
              }

param_sampling = BayesianParameterSampling(param_space)

##### 3. Directivas de terminación anticipada para ajuste de hiperparámetros

**Problema**: En espacios de búsqueda grandes, probar todas las combinaciones de hiperparámetros puede llevar mucho tiempo.

**Solución**: Implementar una directiva de terminación anticipada para abandonar las ejecuciones con bajo rendimiento.

Tipos de directivas:
- **Política de bandidos**: Abandona una ejecución si su rendimiento es peor que el mejor hasta ahora por un margen especificado.
- **Mediana de la política de detención**: Abandona las ejecuciones con un rendimiento peor que la media de todas las ejecuciones.
- **Política de selección de truncamiento**: Cancela un porcentaje X de las ejecuciones con el peor rendimiento en cada intervalo de evaluación.

Parámetros:
- **Intervalo de evaluación**: Frecuencia con la que se evalúa la directiva.
- **Retraso de evaluación**: Número mínimo de iteraciones antes de la primera evaluación.

Beneficios:
- Ahorro de tiempo y recursos computacionales.
- Mayor eficiencia en la búsqueda de los mejores hiperparámetros.

Aplicación:
- Especialmente útil para el aprendizaje profundo, donde el entrenamiento es iterativo.
- El script de entrenamiento puede informar la métrica de rendimiento después de cada época.
- La directiva se puede aplicar después de un número específico de iteraciones.

Ejemplo:
- La directiva se aplica después de la iteración 5.
- Se abandonan las ejecuciones con un rendimiento 0,2 o más peor que la mejor ejecución.

Consideraciones:
- Elegir la política adecuada para el caso de uso específico.
- Ajustar los parámetros de la directiva para optimizar el rendimiento.


Las directivas de terminación anticipada son una herramienta valiosa para acelerar el ajuste de hiperparámetros y mejorar la eficiencia del proceso.

##### Ejecución de un experimento de ajuste de hiperparámetros

En Azure ML, puedes ajustar los hiperparámetros ejecutando un experimento de hiperimpulsor.

- **Creación de un script de entrenamiento para el ajuste de hiperparámetros**

    Para llevar a cabo un experimento de hiperimpulsor, necesitamos crear un script de entrenamiento. Este proceso es similar al que seguiremos en cualquier otro experimento de entrenamiento, con la diferencia de que el script debe:

    - Incluir un argumento para cada hiperparámetro que desees variar.
    - Registrar la métrica de rendimiento objetivo. Esto permite que la ejecución del hiperimpulsor evalúe el rendimiento de las ejecuciones secundarias que inicia e identifique la que produce el modelo de mejor rendimiento.
    
    Por ejemplo, podrías tener un script que entrena un modelo de regresión logística. En este script, usarías un argumento `--regularization` para establecer el hiperparámetro de tasa de regularización y registrarías la métrica de precisión con el nombre `Accuracy`

In [ ]:
import argparse
import joblib
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Obtener el hiparparametro de regularizacion
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01)
args = parser.parse_args()
reg = args.reg_rate

# Obtener el contexto de ejecucion
run = Run.get_context()

# Carga de datos
data = run.input_datasets['training_data'].to_pandas_dataframe()

# Separar los datos en caracteristicas y etiquetas
X = data[['feature1','feature2','feature3','feature4']].values
y = data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Entrenar el modelo
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calcular y registrar la precision
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# guardar el modelo
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model.pkl')

run.complete()

- **Configuración y ejecución de un experimento de hiperimpulsor**

    Para preparar el experimento de hiperimpulsor, debemos usar un objeto `HyperDriveConfig` para configurar la ejecución del experimento

In [ ]:
from azureml.core import Experiment
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal

# Asumiendo ws, script_config y param_sampling definidos previamente
hyperdrive = HyperDriveConfig(run_config=script_config,
                              hyperparameter_sampling=param_sampling,
                              policy=None,
                              primary_metric_name='Accuracy',
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=6,
                              max_concurrent_runs=4)

experiment = Experiment(workspace = ws, name = 'hyperdrive_training')
hyperdrive_run = experiment.submit(config=hyperdrive)

- **Supervisión y revisión de las ejecuciones de hiperimpulsores**

    Puede supervisar los experimentos de hiperimpulsores en Azure ML Studio o mediante el widget `RunDetails` de Jupyter Notebooks  .

    El experimento iniciará una ejecución secundaria para cada combinación de hiperparámetros que se vaya a probar, y puede recuperar las métricas registradas que se ejecutan

In [ ]:
for child_run in run.get_children():
    print(child_run.id, child_run.get_metrics())

También puede enumerar todas las ejecuciones en orden descendente de rendimiento

In [ ]:
for child_run in hyperdrive_run.get_children_sorted_by_primary_metric():
    print(child_run)

Para recuperar la ejecución con mejor rendimiento

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()